# Data Engineering Capstone Project

## Enviroment setup

In [1]:
# Import necessary libraries
import pandas as pd
from pyspark.sql import SparkSession
import configparser

In [2]:
# Read config file
config = configparser.ConfigParser()
config.read_file(open('dl.cfg'))

INPUT_DATA = config['LOCAL']['INPUT_DATA']
INPUT_DATA_VACCINES = config['LOCAL']['INPUT_DATA_VACCINES']
DATA_COLUMNS = config['COMMON']['DATA_COLUMNS']

In [3]:
# Spark session
spark = SparkSession \
        .builder\
        .getOrCreate()

## Step 1: Scope the Project and Gather Data
In this step, we’ll:

* Identify and gather the data we'll be using for our project (at least two sources and more than 1 million rows).
* Explain what end use cases we'd like to prepare the data for (e.g., analytics table, app back-end, source-of-truth database, etc.)

We choose the following datasets:
* Brazilian Government' dataset [COVID-19 population imunization program](https://dados.gov.br/dataset/covid-19-vacinacao/resource/ef3bd0b8-b605-474b-9ae5-c97390c197a8?inner_span=True)

In [4]:
vaccines_df = spark.read.csv(INPUT_DATA_VACCINES, sep=';', header=True)

## Step 2: Explore and Assess the Data
In this step we need:
* Explore the data to identify data quality issues, like missing values, duplicate data, etc.
* Document steps necessary to clean the data

In [5]:
# Read the data dictionary from JSON and extract the valid columns
col_names = pd.read_json(DATA_COLUMNS, typ='series')
valid_columns = col_names.index.to_list()
valid_columns

['paciente_id',
 'paciente_idade',
 'paciente_datanascimento',
 'paciente_enumsexobiologico',
 'paciente_endereco_nmpais',
 'paciente_endereco_uf',
 'paciente_endereco_nmmunicipio',
 'estabelecimento_razaosocial',
 'estalecimento_nofantasia',
 'estabelecimento_uf',
 'estabelecimento_municipio_nome',
 'vacina_categoria_codigo',
 'vacina_categoria_nome',
 'vacina_grupoatendimento_codigo',
 'vacina_grupoatendimento_nome',
 'vacina_fabricante_nome',
 'vacina_codigo',
 'vacina_nome']

In [6]:
# Get the difference between the dataframe colums and the valid columns
columns_todrop = list(set(vaccines_df.columns) - set(valid_columns))

columns_todrop

['document_id',
 'vacina_descricao_dose',
 'sistema_origem',
 'paciente_endereco_coibgemunicipio',
 'vacina_dataaplicacao',
 'id_sistema_origem',
 'paciente_endereco_cep',
 'paciente_racacor_codigo',
 'paciente_endereco_copais',
 'estabelecimento_valor',
 'data_importacao_rnds',
 'paciente_racacor_valor',
 'estabelecimento_municipio_codigo',
 'paciente_nacionalidade_enumnacionalidade',
 'vacina_lote',
 'vacina_fabricante_referencia']

In [7]:
# Remove unused columns from dataframe
vaccines_df = vaccines_df.drop(*columns_todrop)
vaccines_df.printSchema()

root
 |-- paciente_id: string (nullable = true)
 |-- paciente_idade: string (nullable = true)
 |-- paciente_datanascimento: string (nullable = true)
 |-- paciente_enumsexobiologico: string (nullable = true)
 |-- paciente_endereco_nmmunicipio: string (nullable = true)
 |-- paciente_endereco_nmpais: string (nullable = true)
 |-- paciente_endereco_uf: string (nullable = true)
 |-- estabelecimento_razaosocial: string (nullable = true)
 |-- estalecimento_nofantasia: string (nullable = true)
 |-- estabelecimento_municipio_nome: string (nullable = true)
 |-- estabelecimento_uf: string (nullable = true)
 |-- vacina_grupoatendimento_codigo: string (nullable = true)
 |-- vacina_grupoatendimento_nome: string (nullable = true)
 |-- vacina_categoria_codigo: string (nullable = true)
 |-- vacina_categoria_nome: string (nullable = true)
 |-- vacina_fabricante_nome: string (nullable = true)
 |-- vacina_codigo: string (nullable = true)
 |-- vacina_nome: string (nullable = true)



In [8]:
# Replace the null values
vaccines_df = vaccines_df.fillna({'vacina_categoria_codigo': -1, 'vacina_categoria_nome': 'N/A', 'vacina_grupoatendimento_nome': 'N/A'})

In [12]:
vaccines_df.select("vacina_grupoatendimento_nome", "vacina_grupoatendimento_codigo").show()

+----------------------------+------------------------------+
|vacina_grupoatendimento_nome|vacina_grupoatendimento_codigo|
+----------------------------+------------------------------+
|        Pessoas de 18 a 6...|                        000201|
|        Pessoas de 18 a 6...|                        000201|
|        Pessoas de 18 a 6...|                        000201|
|        Pessoas de 18 a 6...|                        000201|
|        Pessoas de 18 a 6...|                        000201|
|                      Outros|                        000926|
|        Pessoas de 18 a 6...|                        000201|
|        Pessoas de 70 a 7...|                        000203|
|        Pessoas de 18 a 6...|                        000201|
|        Pessoas de 18 a 6...|                        000201|
|        Pessoas de 18 a 6...|                        000201|
|        Pessoas de 65 a 6...|                        000202|
|           Diabetes Mellitus|                        000103|
|       

In [ ]:
vaccines_df.select("*").filter(vaccines_df.estabelecimento_municipio_nome == 'PINHEIRO').show(10)